# Cross validation using general function

Examples for 
http://madlib.apache.org/docs/latest/group__grp__validation.html

In [1]:
%load_ext sql

/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
# Greenplum Database 5.4.0 on GCP (demo machine)
#%sql postgresql://gpadmin@35.184.232.200:5432/madlib
        
# PostgreSQL local
%sql postgresql://fmcquillan@localhost:5432/madlib

u'Connected: fmcquillan@madlib'

In [3]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.15.1, git revision: rc/1.15.1-rc1, cmake configuration time: Wed Oct 10 04:29:25 UTC 2018, build type: Release, build system: Darwin-17.7.0, C compiler: Clang, C++ compiler: Clang"


# 1.  Create data set
House prices and characteristics.

In [15]:
%%sql
DROP TABLE IF EXISTS houses;

CREATE TABLE houses ( id INT,
                      tax INT,
                      bedroom INT,
                      bath FLOAT,
                      size INT,
                      lot INT,
                      zipcode INT,
                      price INT,
                      high_priced BOOLEAN
                      );

INSERT INTO houses (id, tax, bedroom, bath, price, size, lot, zipcode, high_priced) VALUES
(1  ,  590 ,       2 ,    1 ,  50000 ,  770 , 22100  , 94301, 'f'::boolean),
(2  , 1050 ,       3 ,    2 ,  85000 , 1410 , 12000  , 94301, 'f'::boolean),
(3  ,   20 ,       3 ,    1 ,  22500 , 1060 ,  3500  , 94301, 'f'::boolean),
(4  ,  870 ,       2 ,    2 ,  90000 , 1300 , 17500  , 94301, 'f'::boolean),
(5  , 1320 ,       3 ,    2 , 133000 , 1500 , 30000  , 94301, 't'::boolean),
(6  , 1350 ,       2 ,    1 ,  90500 ,  820 , 25700  , 94301, 'f'::boolean),
(7  , 2790 ,       3 ,  2.5 , 260000 , 2130 , 25000  , 94301, 't'::boolean),
(8  ,  680 ,       2 ,    1 , 142500 , 1170 , 22000  , 94301, 't'::boolean),
(9  , 1840 ,       3 ,    2 , 160000 , 1500 , 19000  , 94301, 't'::boolean),
(10 , 3680 ,       4 ,    2 , 240000 , 2790 , 20000  , 94301, 't'::boolean),
(11 , 1660 ,       3 ,    1 ,  87000 , 1030 , 17500  , 94301, 'f'::boolean),
(12 , 1620 ,       3 ,    2 , 118600 , 1250 , 20000  , 94301, 't'::boolean),
(13 , 3100 ,       3 ,    2 , 140000 , 1760 , 38000  , 94301, 't'::boolean),
(14 , 2070 ,       2 ,    3 , 148000 , 1550 , 14000  , 94301, 't'::boolean),
(15 ,  650 ,       3 ,  1.5 ,  65000 , 1450 , 12000  , 94301, 'f'::boolean),
(16 ,  770 ,       2 ,    2 ,  91000 , 1300 , 17500  , 76010, 'f'::boolean),
(17 , 1220 ,       3 ,    2 , 132300 , 1500 , 30000  , 76010, 't'::boolean),
(18 , 1150 ,       2 ,    1 ,  91100 ,  820 , 25700  , 76010, 'f'::boolean),
(19 , 2690 ,       3 ,  2.5 , 260011 , 2130 , 25000  , 76010, 't'::boolean),
(20 ,  780 ,       2 ,    1 , 141800 , 1170 , 22000  , 76010, 't'::boolean),
(21 , 1910 ,       3 ,    2 , 160900 , 1500 , 19000  , 76010, 't'::boolean),
(22 , 3600 ,       4 ,    2 , 239000 , 2790 , 20000  , 76010, 't'::boolean),
(23 , 1600 ,       3 ,    1 ,  81010 , 1030 , 17500  , 76010, 'f'::boolean),
(24 , 1590 ,       3 ,    2 , 117910 , 1250 , 20000  , 76010, 'f'::boolean),
(25 , 3200 ,       3 ,    2 , 141100 , 1760 , 38000  , 76010, 't'::boolean),
(26 , 2270 ,       2 ,    3 , 148011 , 1550 , 14000  , 76010, 't'::boolean),
(27 ,  750 ,       3 ,  1.5 ,  66000 , 1450 , 12000  , 76010, 'f'::boolean),
(28 , 2690 ,       3 ,  2.5 , 260011 , 2130 , 25000  , 76010, 't'::boolean),
(29 ,  780 ,       2 ,    1 , 141800 , 1170 , 22000  , 76010, 't'::boolean),
(30 , 1910 ,       3 ,    2 , 160900 , 1500 , 19000  , 76010, 't'::boolean),
(31 , 3600 ,       4 ,    2 , 239000 , 2790 , 20000  , 76010, 't'::boolean),
(32 , 1600 ,       3 ,    1 ,  81010 , 1030 , 17500  , 76010, 'f'::boolean),
(33 , 1590 ,       3 ,    2 , 117910 , 1250 , 20000  , 76010, 'f'::boolean),
(34 , 3200 ,       3 ,    2 , 141100 , 1760 , 38000  , 76010, 't'::boolean),
(35 , 2270 ,       2 ,    3 , 148011 , 1550 , 14000  , 76010, 't'::boolean),
(36 ,  750 ,       3 ,  1.5 ,  66000 , 1450 , 12000  , 76010, 'f'::boolean);

SELECT * FROM houses ORDER BY id;

Done.
Done.
36 rows affected.
36 rows affected.


id,tax,bedroom,bath,size,lot,zipcode,price,high_priced
1,590,2,1.0,770,22100,94301,50000,False
2,1050,3,2.0,1410,12000,94301,85000,False
3,20,3,1.0,1060,3500,94301,22500,False
4,870,2,2.0,1300,17500,94301,90000,False
5,1320,3,2.0,1500,30000,94301,133000,True
6,1350,2,1.0,820,25700,94301,90500,False
7,2790,3,2.5,2130,25000,94301,260000,True
8,680,2,1.0,1170,22000,94301,142500,True
9,1840,3,2.0,1500,19000,94301,160000,True
10,3680,4,2.0,2790,20000,94301,240000,True


# 2. Elastic net

Note that elastic net also has a built in cross validation function for selecting elastic net control parameter alpha and regularization value lambda
http://madlib.apache.org/docs/latest/group__grp__elasticnet.html

But here we use the general function to explore lambda values:

In [13]:
%%sql
DROP TABLE IF EXISTS houses_cv_results;

SELECT madlib.cross_validation_general(
    -- modelling_func
      'madlib.elastic_net_train',
    
    -- modelling_params
        '{%data%, %model%, price, "array[tax, bath, size]", gaussian, 0.5, lambda, TRUE, NULL, fista,
          "{eta = 2, max_stepsize = 2, use_active_set = t}",
          NULL, 10000, 1e-6}'::varchar[],
    
    -- modelling_params_type
        '{varchar, varchar, varchar, varchar, varchar, double precision,
          double precision, boolean, varchar, varchar, varchar, varchar,
          integer, double precision}'::varchar[],
    
    -- param_explored
      'lambda',
    
    -- explore_values
      '{0.1, 0.2}'::varchar[],
    
    -- predict_func
      'madlib.elastic_net_predict',
    
    -- predict_params
        '{%model%, %data%, %id%, %prediction%}'::varchar[],
    
    -- predict_params_type
        '{text, text, text, text}'::varchar[],
    
    -- metric_func
      'madlib.mse_error',
    
    -- metric_params
        '{%prediction%, %data%, %id%, price, %error%}'::varchar[],
    
    -- metric_params_type
        '{varchar, varchar, varchar, varchar, varchar}'::varchar[],
    
    -- data_tbl
      'houses',
    
    -- data_id
      'id',
    
    -- id_is_random
      FALSE,
    
    -- validation_result
      'houses_cv_results',
    
    -- data_cols
      NULL,
    
    -- fold_num
      3
);

Done.
1 rows affected.


cross_validation_general
""


In [14]:
%%sql
SELECT * FROM houses_cv_results;

2 rows affected.


lambda,mean_squared_error_avg,mean_squared_error_stddev
0.1,1194685622.16,366687470.78
0.2,1181768409.98,352203200.758


# 3. Logistic regression 

Here we use the general function to explore maximum number of iterations:

In [20]:
%%sql
DROP TABLE IF EXISTS houses_logregr_cv;

SELECT madlib.cross_validation_general(
    -- modelling_func
        'madlib.logregr_train',
    
    -- modelling_params
        '{%data%, %model%, high_priced, "ARRAY[1, bedroom, bath, size]", NULL, max_iter}'::varchar[],
    
    -- modelling_params_type
        '{varchar, varchar, varchar, varchar, varchar, integer}'::varchar[],
    
    -- param_explored
        'max_iter',
    
    -- explore_values
        '{2, 10, 40, 100}'::varchar[],
    
    -- predict_func
        'madlib.cv_logregr_predict',
    
    -- predict_params
        '{%model%, %data%, "ARRAY[1, bedroom, bath, size]", id, %prediction%}'::varchar[],
    
    -- predict_params_type
        '{varchar, varchar,varchar,varchar,varchar}'::varchar[],
    
    -- metric_func
        'madlib.misclassification_avg',
    
    -- metric_params
        '{%prediction%, %data%,  id, high_priced, %error%}'::varchar[],
    
    -- metric_params_type
        '{varchar, varchar, varchar, varchar, varchar}'::varchar[],
    
    -- data_tbl
        'houses',
    
    -- data_id
        'id',
    
    -- id_is_random
        FALSE,
    
    -- validation_result
        'houses_logregr_cv',
    
    -- data_cols
        NULL,
    
    -- fold_num
       5
);

Done.
1 rows affected.


cross_validation_general
""


In [21]:
%%sql
SELECT * FROM houses_logregr_cv;

4 rows affected.


max_iter,error_rate_avg,error_rate_stddev
2,0.19642857142857142857,0.0818317088384971429780598253843971801653
10,0.22142857142857142857,0.0731925054711399884549944979733273803475
40,0.22142857142857142857,0.0731925054711399884549944979733273803475
100,0.22142857142857142857,0.0731925054711399884549944979733273803475


# 4. Decision tree

Here we use the general function to explore tree depth.  First we need to create a wrapper function for predict that does a column rename:

In [23]:
%%sql
CREATE OR REPLACE FUNCTION tree_predict_rename_col(model_table VARCHAR, data_table VARCHAR, output_table VARCHAR,
                                                   orig_column VARCHAR, new_column VARCHAR)
RETURNS VOID AS $$
BEGIN
    EXECUTE format('SELECT madlib.tree_predict(''%s'', ''%s'', ''%s'')', model_table, data_table, output_table);
    EXECUTE 'ALTER TABLE ' || output_table || ' RENAME ' || orig_column || ' TO ' || new_column;
END
$$ LANGUAGE plpgsql VOLATILE;

Done.


[]

In [24]:
%%sql
DROP TABLE IF EXISTS houses_dt_cv;

SELECT madlib.cross_validation_general(
    -- modelling_func
        'madlib.tree_train',
    
    -- modelling_params
        '{%data%, %model%, id, high_priced, "bedroom, bath, size, zipcode", NULL, NULL, NULL, NULL, max_depth, 1, 1, 10}'::varchar[],
    
    -- modelling_params_type
        '{varchar, varchar, varchar, varchar, varchar, varchar, varchar, varchar, varchar, integer, integer, integer, integer}',
    
    -- param_explored
        'max_depth',
    
    -- explore_values
        '{1, 2, 3, 4}'::varchar[],
    
    -- predict_func
        'tree_predict_rename_col',
    
    -- predict_params
        '{%model%, %data%, %prediction%, estimated_high_priced, prediction}'::varchar[],
    
    -- predict_params_type
        '{varchar,varchar,varchar,varchar,varchar}'::varchar[],
    
    -- metric_func
        'madlib.misclassification_avg',
    
    -- metric_params
        '{%prediction%, %data%,  id, high_priced, %error%}'::varchar[],
    
    -- metric_params_type
        '{varchar, varchar, varchar, varchar, varchar}'::varchar[],
    
    -- data_tbl
        'houses',
    
    -- data_id
        'id',
    
    -- id_is_random
        FALSE,
    
    -- validation_result
        'houses_dt_cv',
    
    -- data_cols
        NULL,
    
    -- fold_num
       5
);
SELECT * FROM houses_dt_cv;

Done.
1 rows affected.
4 rows affected.


max_depth,error_rate_avg,error_rate_stddev
1,0.16785714285714285714,0.1208494593977759334440468761527971235643
2,0.16785714285714285714,0.1208494593977759334440468761527971235643
3,0.17142857142857142857,0.1564921592871903181329101774752513216155
4,0.14285714285714285714,0.1428571428571428571449999999999999999999
